In [1]:
import numpy as np
from fakespikes import neurons, util, rates
from fakespikes.rates import osc2
from fakespikes.neurons import Spikes

from pacological2.neurons import stim
from pacological2.util import rate_code

In [2]:
# Gen stim, and random noise, and gain-pac
# S - stim pop
t = 1.2
time_step = 1e-3
times = util.create_times(t, time_step)

f = 10

scale = 1e2
rate = 30
rate_noise = 10


min_rate = 20
max_rate = 30
g_min = 0.05
g_max = 0.2
back_rate_e = 22
back_rate_i = 22

stim_seed = 10  # Fix this for all exps
N_stim = 400
N_pac = N_stim

# Rate gen
s_rates, ns_s, ts_s = stim(
    t,
    N_stim,
    rate=rate,
    scale=scale,
    rate_noise=rate_noise,
    time_step=time_step,
    seed=stim_seed)

o_rates = osc2(times, back_rate_e, f, min_a=min_rate)

# gain / rate
dg = [g_max, g_min]
do = [min_rate, max_rate]
dg_do = np.interp(o_rates, do, dg)

y_rates = s_rates * o_rates * dg_do

# Noise
xi_rates = np.random.uniform(0, 10, size=len(times))

# Spike gen
Ipri = 2  # 2 Hz var in rate / neuron
lnp_s = Spikes(N_stim, t, dt=time_step, private_stdev=Ipri)
lnp_y = Spikes(N_pac, t, dt=time_step, private_stdev=Ipri)
lnp_xi = Spikes(N_pac, t, dt=time_step, private_stdev=Ipri)

# Run LNP
ns_s, ts_s = util.to_spiketimes(times, lnp_s.poisson(s_rates))
ns_y, ts_y = util.to_spiketimes(times, lnp_y.poisson(y_rates))
ns_xi, ts_xi = util.to_spiketimes(times, lnp_y.poisson(xi_rates))

In [3]:
k = 10
integration_w = 1e-3
drop_before = 0.2

r_s = rate_code(ts_s[ts_s > drop_before], (0, t), integration_w, k=k)
r_y = rate_code(ts_y[ts_y > drop_before], (0, t), integration_w, k=k)
r_xi = rate_code(ts_xi[ts_xi > drop_before], (0, t), integration_w, k=k)

In [6]:
util.mi(r_s, r_y)

0.95645752711191623

In [7]:
util.mi(r_s, r_xi)

0.15967000762492578

In [8]:
util.mi(r_y, r_xi)

0.17315791969641481